In [10]:
import plotly.express as px
import pandas as pd
from src.config.database_models import engine
df_orders = pd.read_sql('orders_fact', engine)
df_orders_sazonal = df_orders[['id_order', 'price', 'date_purchase']]

In [11]:
df_orders_sazonal = df_orders_sazonal.groupby(by='date_purchase').agg({
    'price': ['sum']
})
df_orders_sazonal = df_orders_sazonal.reset_index()
df_orders_sazonal.columns = ['dt_year_month', 'sum_values']

In [12]:
fig = px.line(df_orders_sazonal, 
              x='dt_year_month', 
              y='sum_values',
              labels={'dt_year_month': 'Mês-Ano', 'sum_values': 'Faturamento Total'},
              text='dt_year_month')
fig.update_traces(textposition='bottom right')
fig.show()

In [13]:
df_orders_store = df_orders[['id_order', 'id_store', 'price']]
df_orders_store = df_orders_store.groupby('id_store').agg({
    'price': ['sum'],
    'id_order': ['count']
})
df_orders_store = df_orders_store.reset_index()
df_orders_store.columns = ['id_store', 'sum_values', 'count_orders']
df_orders_store

,id_store,sum_values,count_orders
0,LJ-01,148146.81,393
1,LJ-02,146578.48,406
2,LJ-03,154381.59,411
3,LJ-04,150381.52,403
4,LJ-05,130648.08,373
5,LJ-06,143080.62,406
6,LJ-07,156908.79,433
7,LJ-08,147956.84,409
8,LJ-09,159809.03,420
9,LJ-10,154293.39,401


In [14]:
fig = px.bar(df_orders_store.sort_values('sum_values', ascending=False),
             x='id_store',
             y='sum_values',
             labels={'id_store': 'Código da Loja', 'sum_values': 'Faturamento Anual'},)
fig.show()

In [15]:
fig2 = px.bar(df_orders_store.sort_values('count_orders', ascending=False),
             x='id_store',
             y='count_orders',
             labels={'id_store': 'Código da Loja', 'count_orders': 'Qtd. Vendas'},
             color_discrete_sequence=['red'])
fig2.show()

In [16]:
df_geo = pd.read_sql('stores_dim', engine)
df_orders_store = pd.merge(df_orders_store, df_geo, how='left', on='id_store')
df_orders_store_sum = df_orders_store.groupby('city').agg({
    'sum_values': ['sum'],
    'count_orders': ['sum']
}).reset_index()
df_orders_store_sum.columns = ['cidade', 'faturamento', 'qtdvendas']
df_orders_store_sum['ticket_medio'] = df_orders_store_sum['faturamento'] / df_orders_store_sum['qtdvendas']

df_orders_store_sum = df_orders_store_sum[['cidade', 'ticket_medio']]
df_orders_store_sum

,cidade,ticket_medio
0,Belo Horizonte,369.102558
1,Rio de Janeiro,363.746932
2,São Paulo,370.392172
